In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import math,time,os

In [ ]:
def parse_label(path):
    raw = open(path).read()
    classnames =[]
    for cls in raw.split('\n'):
        if cls == '':
            continue
        classnames.append(cls)
    return classnames

def _decode_and_resize(filename, label):
    image_string = tf.io.read_file(filename)  # 读取原始文件
    image_decoded = tf.io.decode_jpeg(image_string,channels=3)  # 解码JPEG图片
    image_resized = tf.image.resize(image_decoded, [300, 300]) / 255.0
    return image_resized, label

def load_data(dirs):
    images_name = []
    labels = []
    for i,file_dir in enumerate(dirs):
        filename = tf.constant([file_dir + '/' + filename for filename in os.listdir(file_dir)])
        images_name = tf.concat([images_name,filename],axis=-1)
        labels = tf.concat([labels,tf.constant(i,shape=filename.shape[0])],axis=-1)
    print("total:%d" %  images_name.shape[0])
    train_dataset = tf.data.Dataset.from_tensor_slices((images_name, labels))
    train_dataset = train_dataset.map(
        map_func=_decode_and_resize,
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return train_dataset

num_epochs = 5
batch_size = 5
learning_rate = 0.01
buffer_size = 10000

train_dir = './data/train/'
class_names = parse_label(train_dir+'classes_label.txt')
train_file_dirs = [train_dir+classname for classname in class_names] 
train_dataset = load_data(train_file_dirs)
train_dataset = train_dataset.shuffle(buffer_size)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
class DANN(tf.keras.Model):
    
    def __init__(self,num_class):
        super(DANN, self).__init__()
        self.base = tf.keras.applications.EfficientNetB3(classifier_activation=None)
        self.label1 = tf.keras.layers.Dense(256, activation=tf.keras.layers.LeakyReLU())
        self.label2 = tf.keras.layers.Dense(40, activation=tf.keras.layers.Softmax())
#         self.domain1 = tf.keras.layers.Dense(256, activation=tf.keras.layers.LeakyReLU())
#         self.domain2 = tf.keras.layers.Dense(1, activation=tf.keras.layers.Softmax())
        
    def call(self, inputs):
        label = self.base(inputs)
        label = self.label1(label)
        label = self.label2(label)
#         domain = self.base(inputs)
#         domain = self.domain1(domain)
#         domain = self.domain2(domain)
        return label #,domain

In [ ]:
model = DANN(len(class_names))
model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        metrics=[tf.keras.metrics.sparse_categorical_accuracy]
    )
model.fit(train_dataset,epochs=num_epochs)
model.save('model/dann-en3-'+time.strftime("%m-%d-%H-%M", time.localtime())+'.h5')